In [1]:
import circuitsvis as cv
import numpy as np
import torch
from taker import Model
from taker.hooks import HookConfig
from datetime import datetime
import json
from os import listdir
from os.path import exists

import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import torch

/workspace/SPAR/interp-ab/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"../results/{current_time}_agnes_story_singlenewline.jsonl"

In [3]:
m = Model("microsoft/phi-3-mini-4k-instruct", dtype="int4")
m.show_details()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.50s/it]
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
You are not running the flash-attention implementation, expect numerical differences.


Loaded model 'microsoft/phi-3-mini-4k-instruct' with int4:
- Added 512 hooks across 32 layers


/workspace/SPAR/interp-ab/.venv/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


 - n_layers : 32
 - d_model  : 3072
 - n_heads  : 32
 - d_head   : 96
 - d_mlp    : 8192


In [9]:
prompt    = """Write a short blog post about a recipe and the inspiration behind it.
 Do not include a title.
 Only reveal the dish after the story.
 Start with short story and then move to the recipe.
 To re-iterate, do not include a title."""
# info_gen = m.generate(info_prompt, temperature=0.3, num=300)

#DOUBLE NEWLINE
info_gen = """
\n Once upon a time, in a quaint little village nestled between rolling hills and verdant fields, there lived an elderly woman named Agnes. Agnes was known for her warm smile and her legendary Sunday dinners that brought the entire neighborhood together. Her recipes were family heirlooms, passed down through generations, with each family adding their own touch to the final dish.

One crisp autumn evening, Agnes was reminiscing about her childhood, and how her grandmother used to gather everyone around the dinner table, sharing stories and laughter. These were the moments that shaped her, the memories that she passed on to her own children and grandchildren.

Inspired by her grandmother's legacy, Agnes decided to create a new dish that would encapsulate the essence of those cherished gatherings. She wanted something that was comforting and nourishing, a dish that could be prepared with love and shared with others. After days of experimentation, she finally created a recipe that she believed truly captured the spirit of her family's Sunday dinners.\n\n"""
info_prompt = prompt+info_gen

In [10]:
import sys, os
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [11]:
idlist = m.get_ids(".\n\n").squeeze().tolist()
print(m.tokenizer.convert_ids_to_tokens(idlist))
idlist = m.get_ids("Sunday dinners.\n\n").squeeze().tolist()
print(m.tokenizer.convert_ids_to_tokens(idlist))

['▁.', '<0x0A>', '<0x0A>']
['▁Sunday', '▁d', 'inners', '.', '<0x0A>', '<0x0A>']


In [12]:
if not exists(filename):
    with open(filename, "w") as f:
        pass

neutral_prompt = "\n"
max_new_tokens = 100
temperature = 0.3
tokens_transferred_num = 1

acts = m.get_midlayer_activations(info_prompt)
orig_token_index = m.get_ids(info_prompt).shape[1] - 1
new_token_index  = m.get_ids(neutral_prompt).shape[1] - 1        
print(orig_token_index, new_token_index)
print(m.tokenizer.convert_ids_to_tokens(m.get_ids(info_prompt).squeeze().tolist()[298:304]))
print(m.tokenizer.convert_ids_to_tokens(m.get_ids(neutral_prompt).squeeze().tolist()))

for token_val in range(tokens_transferred_num):
    [h.reset() for h in m.hooks.neuron_replace.values()] #RESET HOOKS BEFORE TRANSPLANTING NEXT SET OF ACTIVATIONS
    for j in range(token_val + 1):
        for layer_index in range(0,32):
            m.hooks.neuron_replace[f"layer_{layer_index}_mlp_pre_out"].add_token(new_token_index - j, acts["mlp"][0, layer_index, orig_token_index - j])
            m.hooks.neuron_replace[f"layer_{layer_index}_attn_pre_out"].add_token(new_token_index - j, acts["attn"][0, layer_index, orig_token_index - j])
    with HiddenPrints():
        for i in range(20):
            output = m.generate(neutral_prompt, max_new_tokens, temperature=temperature)
            
            data = {
                "temperature": temperature,
                "max_new_tokens": max_new_tokens,
                "model": "microsoft/phi-3-mini-4k-instruct",
                "transplant_layers": (0,32),
                "transferred_token_num": token_val+1,
                "orig_prompt": info_prompt,
                "transplant_prompt": neutral_prompt,
                "output": output[1],
            }

            with open(filename, "a") as file:
                file.write(json.dumps(data) + "\n")

303 1
['▁Sunday', '▁d', 'inners', '.', '<0x0A>', '<0x0A>']
['▁', '<0x0A>']


In [18]:

filepath2 = "../results/2024-08-15_14-38-54_agnes_story_sundaydinners.jsonl"
df1 = pd.read_json(filepath2, lines=True)
filepath2 = "../results/2024-08-15_14-49-58_agnes_story_dinners.jsonl"
df2 = pd.read_json(filepath2, lines=True)
filepath2 = "../results/2024-08-15_14-59-11_agnes_story_inners.jsonl"
df3 = pd.read_json(filepath2, lines=True)
filepath1 = "../results/2024-08-08_15-27-40_agnes_story_period.jsonl"
df4 = pd.read_json(filepath1, lines=True)
filepath2 = "../results/2024-08-15_15-08-26_agnes_story_doublenewline.jsonl"
df5 = pd.read_json(filepath2, lines=True)
filepath2 = "../results/2024-08-15_15-12-10_agnes_story_singlenewline.jsonl"
df6 = pd.read_json(filepath2, lines=True)
results_df = pd.concat([df1, df2, df3, df4, df5, df6],ignore_index=True)
print(results_df.loc[0])
baselinefile = "../results/baseline_data_08-07.jsonl"
base_df = pd.read_json(baselinefile, lines=True)
neutralfile = "../results/neutral_baseline_data_08-14.jsonl"
neutral_df = pd.read_json(neutralfile, lines=True)
results_df.head()

temperature                                                            0.3
max_new_tokens                                                         100
model                                     microsoft/phi-3-mini-4k-instruct
transplant_layers                                                  [0, 32]
transferred_token_num                                                    1
orig_prompt              Write a short blog post about a recipe and the...
transplant_prompt                                      Sunday dinners.\n\n
output                   The recipe calls for 2 teasquant of olive oil,...
Name: 0, dtype: object


,temperature,max_new_tokens,model,transplant_layers,transferred_token_num,orig_prompt,transplant_prompt,output
0,0.3,100,microsoft/phi-3-mini-4k-instruct,"[0, 32]",1,Write a short blog post about a recipe and the...,Sunday dinners.\n\n,"The recipe calls for 2 teasquant of olive oil,..."
1,0.3,100,microsoft/phi-3-mini-4k-instruct,"[0, 32]",1,Write a short blog post about a recipe and the...,Sunday dinners.\n\n,"The recipe for the traditional dish, which is ..."
2,0.3,100,microsoft/phi-3-mini-4k-instruct,"[0, 32]",1,Write a short blog post about a recipe and the...,Sunday dinners.\n\n,The recipe calls for 1/4 cup of oil and 2 cups...
3,0.3,100,microsoft/phi-3-mini-4k-instruct,"[0, 32]",1,Write a short blog post about a recipe and the...,Sunday dinners.\n\n,Here's how to make it:\n\nIngredients:\n\n- 2 ...
4,0.3,100,microsoft/phi-3-mini-4k-instruct,"[0, 32]",1,Write a short blog post about a recipe and the...,Sunday dinners.\n\n,"Again, I'm not sure how to proceed. I'm not su..."


In [19]:
# sentenceTransformer = SentenceTransformer("all-MiniLM-L6-v2")
sentenceTransformer = SentenceTransformer("all-mpnet-base-v2")

concat_embs = results_df["output"].to_list() + base_df["output"].to_list() + neutral_df["output"].to_list()
embeddings = sentenceTransformer.encode(concat_embs)
print(embeddings.shape)
neutral_start = results_df.shape[0]+base_df.shape[0]
results_df["embeddings"] = embeddings[:results_df.shape[0]].tolist()
base_df["embeddings"] = embeddings[results_df.shape[0]:neutral_start].tolist()
neutral_df["embeddings"] = embeddings[neutral_start:].tolist()
results_df.head()
print(results_df.shape)
print(base_df.shape)
print(neutral_df.shape)

(570, 768)
(420, 9)
(50, 6)
(100, 6)


In [35]:
# carry out pca on embeddings using sklearn
reducer = PCA(n_components=2)
embeddings_reduced = reducer.fit_transform(embeddings)
print(embeddings_reduced.shape)

# visualize the embeddings using plotly
import plotly.express as px
import plotly.graph_objects as go

results_df["dim1"] = embeddings_reduced[:results_df.shape[0], 0]
results_df["dim2"] = embeddings_reduced[:results_df.shape[0], 1]
base_df["dim1"] = embeddings_reduced[results_df.shape[0]:neutral_start, 0]
base_df["dim2"] = embeddings_reduced[results_df.shape[0]:neutral_start, 1]
baseline_mean_x = base_df["dim1"].mean(axis=0)
baseline_mean_y = base_df["dim2"].mean(axis=0)
neutral_df["dim1"] = embeddings_reduced[neutral_start:, 0]
neutral_df["dim2"] = embeddings_reduced[neutral_start:, 1]
neutral_mean_x = neutral_df["dim1"].mean(axis=0)
neutral_mean_y = neutral_df["dim2"].mean(axis=0)

results_df["truncated_output"] = results_df["output"].str.slice(0,100)
results_df["transplant_prompt_str"] = results_df["transplant_prompt"].str.replace("\n", "\\n")

fig = px.scatter(
    results_df,
    x="dim1",
    y="dim2",
    hover_data=["transplant_prompt_str", "transferred_token_num"],
    title="PCA of Agnes Story Embeddings",
    color="transferred_token_num",
    symbol="transplant_prompt_str",
    symbol_sequence = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'pentagon', 'hexagon', 'octagon', 'star'],

)


fig.add_trace(go.Scatter(x=base_df["dim1"].to_list(), y=base_df["dim2"].to_list(), mode='markers', marker=dict(color='green', size=10), name='Baseline Generation'))


fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=100, xref="container"))

fig.update_traces(
    hovertemplate="<br>".join(
        ["Transplanted Prompt: %{customdata[0]}", "# of Transferred Tokens: %{customdata[1]}"]
    )
)

fig.add_trace(go.Scatter(x=neutral_df["dim1"].to_list(), y=neutral_df["dim2"].to_list(), mode='markers', marker=dict(color='turquoise', size=8), name='Untransferred Newline'))

fig.data = [fig.data[7],fig.data[6],fig.data[5],fig.data[4], fig.data[3],fig.data[2],fig.data[1],fig.data[0]]


fig.show()

(570, 2)


In [34]:
pca_df = pd.DataFrame(embeddings_reduced, columns=['PC1', 'PC2'])
print(pca_df.head())
print(results_df['transplant_prompt_str'][0])
pca_df['transplant_prompt_str'] = results_df['transplant_prompt_str']
pca_df['transferred_token_num'] = results_df['transferred_token_num']

mean_pca_df = pca_df.groupby(['transplant_prompt_str', 'transferred_token_num']).mean().reset_index()
print(mean_pca_df.head())

# fig.add_trace(px.scatter(mean_pca_df, x='PC1', y='PC2', color='temperature', symbol='layers_string', 
#                          title='Means of PCA Groups').data[0]):
mean_pca_df['type'] = 'Mean'
pca_df['type'] = 'Original'
combined_df = pd.concat([pca_df, mean_pca_df], ignore_index=True)
# fig = px.scatter(combined_df, x='PC1', y='PC2', color='temperature', symbol='layers_string', title='PCA of Sentence Embeddings')

# Plot mean points
mean_fig = px.scatter(mean_pca_df, x='PC1', y='PC2', color='transferred_token_num', symbol='transplant_prompt_str', title='Means of PCA Groups', category_orders={"transplant_prompt_str":["\n","\n\n",".\n\n","inners.\n\n"," dinners.\n\n","Sunday dinners.\n\n"]}, symbol_sequence = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'pentagon', 'hexagon', 'octagon', 'star'])
    
mean_fig.add_trace(go.Scatter(x=[baseline_mean_x], y=[baseline_mean_y], mode='markers', marker=dict(color='green', size=10), name='Baseline Generation Mean'))

mean_fig.add_trace(go.Scatter(x=[neutral_mean_x], y=[neutral_mean_y], mode='markers', marker=dict(color='turquoise', size=8), name='Untransferred Newline Mean'))
fig.data=[fig.data[3], fig.data[2], fig.data[1], fig.data[0]]

mean_fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=100, xref="container"))


mean_fig.show()

        PC1       PC2
0  0.088585  0.341144
1  0.435141 -0.069570
2 -0.032514  0.429647
3  0.404933  0.243209
4 -0.268462  0.010027
Sunday dinners.\n\n
  transplant_prompt_str  transferred_token_num       PC1       PC2
0          dinners.\n\n                    1.0 -0.083093  0.237141
1          dinners.\n\n                    2.0 -0.082674  0.213721
2          dinners.\n\n                    3.0  0.211505  0.008649
3          dinners.\n\n                    4.0  0.020320 -0.033381
4          dinners.\n\n                    5.0  0.301683 -0.124812
